In [1]:
from app import db
from app.models import *


/usr/local/lib/python3.6/dist-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [8]:
org =  Organisation
org_1 = org.query.all()[0]
org_1.name = "JAICA"
org_2 = org(name='WHAVE')
db.session.add(org_2)
db.session.add(org_1)
db.session.commit()
#print(org_1.name)

In [22]:
usr = User(user_name="jaica", password="jaica123", first_name="jaica")
db.session.add(usr)
db.session.commit()


In [3]:
usr = User.query.get(2)
print("ORG Name --> {}".format(usr.org.name))
print("User Name --> {}".format(usr.user_name))


ORG Name --> WHAVE
User Name --> adam


In [4]:
usr.is_sunda_admin()

False

In [5]:
usr.has_permissions(Permission.ORG_ATTENDANT)

True

In [9]:
Role.insert_roles()

done ---> [ ORG_ADMIN ]
done ---> [ SUNDA_ADMIN ]
done ---> [ ORG_ATTENDANT ]
Done adding All


In [31]:
fetches = Fetches.query.filter(Fetches.volume_taken > 0).order_by(Fetches.id.desc()).paginate(1,500,False).items

In [33]:
fetches = Fetches.query.filter(Fetches.volume_taken > 0).order_by(Fetches.id.desc()).paginate(1,5,False).items

In [41]:
fetches = Fetches.query.filter().paginate(1,500,False).items

In [57]:
fetch_list = []

for f in fetches:
    fetch_list.append(f)
   # print(str(f.card_user.name) +" --> " + str(f.volume_taken))

# fetch_list = list(reversed(fetch_list))

fetch_list = fetch_list[::-1] # this is the same waty as reversing the order of a python list

fetch_list = fetch_list[0:3]

print(len(fetch_list))

for item in fetch_list:
    print(item.card_user.name)
    
    
    

3
Master Card
Master Card
TEST-CARD USER


In [49]:
fetch_count = Fetches.query.filter().count()
print (fetch_count/500)

83.97


In [51]:
kk = int(fetch_count/500)
print(kk-5)

78


In [12]:
Role.insert_roles()

done ---> [ ORG_ADMIN ]
done ---> [ SUNDA_ADMIN ]
done ---> [ ORG_ATTENDANT ]
Done adding All


In [10]:
r = Role.query.get(1)
print(r.name)

Admin


In [3]:
org_service  = OrganisationService()
org_service.insert_service_access()

Updating Service Access --> LLP_SERVICE 
Updating Service Access --> BASIC_PLAN 
Updating Service Access --> BASIC_AND_LLP 


In [5]:
org = Organisation.query.get(2)
print (org.name)

WHAVE


In [6]:
org.has_llp()

True

In [7]:
org.has_basic_plan()

False

In [9]:
## processing Batch data for basic plan or standard plan

_rfid_no = 90143104

card = RFID.query.filter_by(rfid_no=_rfid_no).first()

batch_list = []

batch = db.session.query(
                            MemorySlot.index ,
                            VolumeUpdates.amount,
                            db.func.max(VolumeUpdates.id),
                            RFID.rfid_no
                        )\
                        .join(CardUser, CardUser.id == MemorySlot.card_user_id)\
                        .join(RFID, RFID.user_id == CardUser.id)\
                        .join(Borehole, Borehole.id == RFID.borehole_id)\
                        .join(VolumeUpdates,VolumeUpdates.card_user_id == CardUser.id )\
                        .group_by(CardUser.id)\
                        .filter(
                                Borehole.id == 3,
                                VolumeUpdates.bore_updated == False,
                                )\
                        .all()


for data in batch:
    cv_batch_item = {'s':data[0],'u':data[1],'rfid':data[3]} # s---> slot & c ---> Cumulative value
    batch_list.append(cv_batch_item)
#     print(cv_batch_item)
    
print(batch_list)


[]


In [35]:
basic_batch_list = []

basic_batch = VolumeUpdates.query\
                                .join(CardUser, VolumeUpdates.card_user_id == CardUser.id )\
                                .join(RFID, RFID.user_id == CardUser.id)\
                                .join(Borehole, Borehole.id == RFID.borehole_id)\
                                .filter(
                                    Borehole.id == 3,
                                    VolumeUpdates.bore_updated == False,
                                )

for data in basic_batch:
    
     # s---> slot & u ---> updated value
        
    update_batch_item = {
                         's':data.card_user.memory_slot[0].index,
                         'u':data.amount,
                        } 

    data.bore_updated = not data.bore_updated           
    basic_batch_list.append(update_batch_item)


db.session.commit()
    
print(basic_batch_list)

[]


In [12]:
credit_batch_list = [] 

credit_updates = CreditUpdate.query\
                                    .join(LLPUserAccount, LLPUserAccount.id ==  CreditUpdate.llp_acc_id)\
                                    .join(CardUser, CardUser.id == LLPUserAccount.card_user_id)\
                                    .join(RFID, RFID.user_id == CardUser.id)\
                                    .join(Borehole, Borehole.id == RFID.borehole_id)\
                                    .filter(
                                        CreditUpdate.bore_update == 0,\
                                        Borehole.id == card.borehole_id
                                    )

print(type(credit_updates))

<class 'flask_sqlalchemy.BaseQuery'>


In [2]:
# SQLAlchemy JSON query Builder

from sqlalchemy_json_querybuilder.querybuilder.search import Search

In [92]:
filter_by = []

criteria_1 = {
    'field_name':'CardUser.short_id',
    'operator':'contains',
    'field_value':"001004"
}

filter_by.append(criteria_1)


order_by = ['-CardUser.id']

results = Search(db.session,
                 "app.models",
                 (CardUser,),
                 filter_by = filter_by,
                 order_by=order_by,
                 page=1, per_page=4).results

print(results['count'])

results['data'][0].name



4


'Fred Mityana Test'

In [18]:
# payment criterias with date ranges

_date_from = '2019-03-01'

date_from = {
    'field_name':'Payment.time_stamp',
    'operator' :'gte',
    'field_value': _date_from
}

date_to = {
    'field_name':'Payment.time_stamp',
    'operator':'lte',
    'field_value':'2019-03-15'
}

short_id = {
    'field_name':'Payment.narrative',
    'operator':'eq',
    'field_value':'056056002', 
}

org_payments_criteria = {
    
    'field_name':'Organisation.id',
    'field_value':1,
    'operator':'eq'
}

filter_by = []

# filter_by.append(date_from)
# filter_by.append(date_to)
# filter_by.append(short_id)

filter_by.append(org_payments_criteria)





order_by = ['-Payment.time_stamp']

results = Search(db.session,
                'app.models',
                (Payment,),
                filter_by =  filter_by,
                order_by=order_by).results




# results['count']
print(results['data'][0].amount)



5000


In [8]:
org_payments_criteria = {
    
    'field_name':'Payment.card_user',
    'operator':'has',
    'field_value':{
        
        'field_name':'CardUser.rfid',
        'operator':'any',
        'field_value':{
            
            'field_name': 'RFID.borehole',
            'operator':'has',
            'field_value':{
                
                'field_name':'Borehole.org',
                'operator':'has',
                'field_value':{
                    
                    'field_name':'Organisation.id',
                    'operator':'eq',
                    'field_value':2
                }
            }
        }
    } 
    
}


# short_id = {
#     'field_name':'Payment.narrative',
#     'operator':'eq',
#     'field_value':'056056002', 
# }


payment_filter_by = []

# payment_filter_by.append(date_from)
# payment_filter_by.append(date_to)

payment_filter_by.append(short_id)

payment_filter_by.append(org_payments_criteria)

order_by = ['-Payment.time_stamp']

results = Search(db.session,
                'app.models',
                (Payment,),
                filter_by =  payment_filter_by,
                order_by=order_by).results


print(results['count'])

results['data'][1].amount

2


1000